In [181]:
import pandas
import numpy
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import xlrd # excel support
import cufflinks as cf

We're working with an A-B sample of Bexar County District Court records accessed on 6/2/2018. This are almost entirely felony charges.

In [47]:
df = pandas.read_excel('bexar_sample.xls')
df.columns

Index([u'CASE-CAUSE-NBR', u'FULL-NAME', u'ALIAS', u'SEX', u'RACE',
       u'BIRTHDATE', u'SID', u'JUDICIAL-NBR', u'ADDR-HOUSE-NBR', u'HOUSE-SUF',
       u'ADDR-PRE-DIRECTION', u'ADDR-STREET', u'ADDR-STREET-SUFFIX',
       u'ADDR-POST-DIRECTION', u'ADDR-UNIT', u'ADDR-CITY', u'ADDR-STATE',
       u'ADDR-ZIP-CODE', u'ADDR-ZIP-PLUS-4', u'OFFENSE-DATE', u'OFFENSE-CODE',
       u'OFFENSE-DESC', u'OFFENSE-TYPE', u'REDUCED-OFFENSE-CODE',
       u'REDUCED-OFFENSE-DESC', u'REDUCED-OFFENSE-TYPE', u'LOCATION',
       u'CUSTODY-DATE', u'COMPLAINT-DATE', u'FILING-AGENCY-DESCRIPTION',
       u'CASE-DATE', u'CASE-DESC', u'SETTING-DATE', u'SETTING-TYPE',
       u'G-JURY-DATE', u'G-JURY-STATUS', u'DISPOSITION-DATE',
       u'DISPOSITION-CODE', u'DISPOSITION-DESC', u'JUDGEMENT-DATE',
       u'JUDGEMENT-CODE', u'JUDGEMENT-DESC', u'SENTENCE-DESC', u'SENTENCE',
       u'SENTENCE-START-DATE', u'SENTENCE-END-DATE', u'FINE-AMOUNT',
       u'COURT-COSTS', u'COURT-TYPE', u'COURT', u'POST-JUDICIAL-FIELD',
       

In [50]:
# Turn sentence columns into something more manageable

def sentence_days(row):
    try:
        sentence = str(row['SENTENCE'])
        hour_days = int(sentence[-3:]) / 24
        days = int(sentence[-5:-3]) * 1
        month_days = int(sentence[-7:-5]) * 30
        years = int(sentence[0: -7]) * 365
    except:
        return 0
    return hour_days + days + month_days + years
    
def og_sentence_days(row):
    try:
        sentence = row['ORIGINAL-SENTENCE']
        year_days = int(sentence[0:3]) * 365
        month_days = int(sentence[5:7]) * 30
        days = int(sentence[-10:-8]) * 1
        hour_days = int(sentence[-5:-2]) / 24
    except:
        return 0
    return hour_days + days + month_days + year_days

df['SENTENCE-DAYS'] = df.apply(lambda row: sentence_days(row), axis=1)
df['ORIGINAL-SENTENCE-DAYS'] = df.apply(lambda row: og_sentence_days(row), axis=1)

df[['SENTENCE', 'SENTENCE-DAYS', 'ORIGINAL-SENTENCE', 'ORIGINAL-SENTENCE-DAYS']].head(10)

,SENTENCE,SENTENCE-DAYS,ORIGINAL-SENTENCE,ORIGINAL-SENTENCE-DAYS
0,20000000,730,002YR00MTH00DYS000HR,730
1,20000000,730,002YR00MTH00DYS000HR,730
2,20000000,730,002YR00MTH00DYS000HR,730
3,30000000,1095,010YR00MOS00DYS000HR,3650
4,0,0,,0
5,20000000,730,002YR00MTH00DYS000HR,730
6,20000000,730,002YR00MTH00DYS000HR,730
7,50000000,1825,005YR00MTH00DYS000HR,1825
8,30000000,1095,003YR00MTH00DYS000HR,1095
9,600000,0,002YR00MTH00DYS000HR,730


In [218]:
sentence_df = df[['OFFENSE-TYPE', 'ORIGINAL-SENTENCE-DAYS', 'RACE', 'SEX']].loc[df['ORIGINAL-SENTENCE-DAYS'] > 0]
sentence_df.columns

Index([u'OFFENSE-TYPE', u'ORIGINAL-SENTENCE-DAYS', u'RACE', u'SEX'], dtype='object')

In [219]:
sentence_grouped = sentence_df.groupby(['OFFENSE-TYPE', 'RACE', 'SEX'], as_index=False).agg(['mean', 'count']).reset_index()
sentence_grouped = sentence_grouped.loc[sentence_grouped['ORIGINAL-SENTENCE-DAYS']['count'] > 100]
sentence_grouped = sentence_grouped.reset_index().drop('index', 1).swaplevel(axis=1).drop('count', 1)
sentence_grouped.columns = sentence_grouped.columns.droplevel()
sentence_grouped

,OFFENSE-TYPE,RACE,SEX,ORIGINAL-SENTENCE-DAYS
0,F1,B,M,4902.798013
1,F1,L,F,3465.494624
2,F1,L,M,4635.724156
3,F1,W,F,3226.963190
4,F1,W,M,4501.660297
5,F2,B,F,2154.496855
6,F2,B,M,2867.770642
7,F2,L,F,2266.885932
8,F2,L,M,2637.511129
9,F2,W,F,2122.194215


In [224]:
#sentence_grouped = sentence_grouped.iplot(kind='bar', filename='cufflinks/grouped-bar-chart')
sentence_pivot = sentence_grouped.pivot_table(columns=['SEX', 'RACE'], index='OFFENSE-TYPE', values='ORIGINAL-SENTENCE-DAYS')
sentence_pivot


iplot(sentence_pivot.iplot(asFigure=True,
                               kind='bar'))